### Naka Rushton Modeling for RCA Data Derived from LTP Task (AttnL and AttnR) 

In [ ]:
# load packages
import numpy as np 
import scipy.io
from scipy.fft import fft, ifft
from scipy.io   import  loadmat
import pandas as pd
import os
import matplotlib.pyplot as plt #import matplotlib as plt
from scipy.optimize import curve_fit 
import seaborn as sns
#import mat73

#### Set Functions; Naka Rushton nd Cost Function

In [ ]:
# Naka-Rushton function: Response as function of luminance contrast
def naka_rushton(I, I0, n, Rmax):
    return Rmax * (I**n) / (I0**n + I**n)
## R(C) = Rmax * C^n / ((C^n)underscore50 + C^n) - 3 PARAMETERS
# 1.) Rmax = maximum response (amplitude of VEP)/ spike rate at which the response saturates
# 2.) C50 = contrast necessary to reach 50% of the maximum response (semisaturation contrast) 
# 3.) n = determines the slope of the curve as well as sharpness of of the nonlinearities at low and high contrasts
## n is determined through an iterative fitting process - try least squares fitting
def cost_function(n, I, R, IO, Rmax):
    predictions = naka_rushton(I,IO,n,Rmax)
    return np.sum((predictions - R)**2)
# function that easurea the difference between model predictions and the actual data

#### Set Path for RCA Data File

In [ ]:
# Main Directory of processed file from MatLab
MainDir = 'D:\\AttnXV3_analysis\\RCA_F1\\RCA\\' # set dir
os.chdir(MainDir)
# Set File Name 
d = os.listdir(MainDir) # list files in dir
FileN = (d[0]) # choose one
file_path = os.path.join(MainDir, FileN) # join paths and prep 2 load
print(file_path) # does path exist ... ?
os.path.exists(file_path) # yes or no

#### Import and Extract Data

In [ ]:
mat_data = scipy.io.loadmat(file_path)
rca = mat_data['rcaResult']['projectedData'][0,0]
rcaData = [rca[i,0] for i in range(rca.shape[0])] # final data output

#### Set Params + Import Single Data Set Just before mass loops

In [ ]:
t = rcaData[16] # 24 x 4 x 78
t[t == 0] = np.nan # replace 0's with nan's
[NumRows, NumComps, NumTrials] = t.shape
NumBins = 6 # number of contrasts
NumHarms = 2 # number of harmonic data: 2F1. 4F1
NumFreqs = (NumRows/NumBins) / 2
FreqBound = int(NumRows/2) # index across cols
TrialBound = int(NumTrials/2) # index across cols
NumComp = 0 # first component from RCA
contrast_levels = np.array([1, 3, 5, 16, 40, 100])
x  = sns.color_palette("Paired",6)

#### Segement Data Based on 4 conditions

In [ ]:
real_pre = np.squeeze (t[0:FreqBound, NumComp, 0:TrialBound]) # 12 x 39
imag_pre = np.squeeze (t[FreqBound:,  NumComp, 0:TrialBound])
real_post = np.squeeze(t[0:FreqBound, NumComp, TrialBound:])
imag_post = np.squeeze(t[FreqBound:,  NumComp, TrialBound:])
print(imag_post.shape) # sanity check

#### Find the Amplitude Per Bin Per harmonic Frequency

In [ ]:
AmpPerBin = np.zeros((2,NumBins*2))
for j in range(12):
    pre = np.hypot(np.nanmean(real_pre[j,:]),np.nanmean(imag_pre[j,:]))
    post = np.hypot(np.nanmean(real_post[j,:]), np.nanmean(imag_post[j,:]))
    AmpPerBin[0,j] = pre
    AmpPerBin[1,j] = post

##### Plot as Sanity Check

In [ ]:
fig, axs = plt.subplots(1, NumHarms, figsize=(12, 6), sharey = True)
#fbins = np.arange(1, 13).reshape(NumHarms, -1)
fbins = np.array([[0, 1, 2, 3, 4, 5,], [6, 7, 8, 9, 10, 11]])

for k in range(NumHarms):
    axs[k].plot(AmpPerBin[0, fbins[k, :]], label='pre')
    axs[k].plot(AmpPerBin[1, fbins[k, :]], label='post')

    axs[k].set_title(f'Subplot {k + 1}')
    axs[k].legend()
plt.show()

#### Stack Pre and Post into a Single Data Array

In [ ]:
crfuns = np.zeros((int(NumFreqs)*2,int(NumBins))) # 4 x 6 array for all crf conditions
# pre / post 2F1
crfuns[0,:] =  AmpPerBin[0,0:6]
crfuns[1,:] =  AmpPerBin[1,0:6]
# pre / post 4F1
crfuns[2,:] =  AmpPerBin[0,6:]
crfuns[3,:] =  AmpPerBin[1,6:]

#### Find C50 and RMax for every Condition

In [ ]:
C50_ind = np.zeros((4))
rMax_ind = np.zeros((4))
for j in range(4):
    C50_ind[j] = contrast_levels[np.argmin(np.abs(crfuns[j,:] - (np.max(crfuns[j,:])/2)))] # contrast that elcits 50% of max neural activity
    rMax_ind[j] = np.max(crfuns[j,:]) # max amplitude before saturation
print(C50_ind)
print(rMax_ind)

In [ ]:
n_optimal = np.zeros((4))

initial_n_guess = 1.0
for iters in range(4):
    OptimalVal, opsCov = curve_fit(lambda I, n: naka_rushton(contrast_levels,n,C50_ind[iters],rMax_ind[iters]), contrast_levels, crfuns[iters,:], p0 = [initial_n_guess],method = 'lm',check_finite = True)
    n_optimal[iters] = OptimalVal[0]
    print('Optimal N:',n_optimal[iters]) # n fit (determines shape of function)
    print(opsCov)
    print(np.linalg.cond(opsCov)) # condition number of covariance matrix
# small number suggests matrix is well conditioned and parameter estimates are stable 
# wrt variations in the input data
    print(np.diag(opsCov)) # should be a matrix .... hmm...

### Plot CRF's and Naka Rushton 

In [ ]:
# Plotting the Naka-Rushton function with the fixed I0 and Rmax, and optimized n
fig, axs = plt.subplots(1, NumHarms, figsize=(14, 6), sharey = True)
# 2F1
axs[0].plot(crfuns[0,:], 'o', markersize = 8, label = 'Pre 2F1', color = x[2])
axs[0].plot(naka_rushton(contrast_levels, n_optimal[0], C50_ind[0], rMax_ind[0]),linewidth=3,label=f'Naka-Rushton, C50={C50_ind[0]:.2f}, Rmax={rMax_ind[0]:.2f}, nFit={n_optimal[0]:.2f}', color = x[2])
axs[0].plot(crfuns[1,:], 'o',markersize = 8,  label = 'Post 2F1', color = x[3])
axs[0].plot(naka_rushton(contrast_levels, n_optimal[1], C50_ind[1], rMax_ind[1]),linewidth=3,label=f'Naka-Rushton, C50={C50_ind[1]:.2f}, Rmax={rMax_ind[1]:.2f}, nFit={n_optimal[1]:.2f}', color = x[3])
axs[0].set_title('2F1 Naka Rushton Sim')
axs[0].legend(fontsize = 10, loc = 'lower right')
axs[0].spines['top'].set_visible(False)
axs[0].spines['right'].set_visible(False)
axs[0].set_ylabel('Response (R)')
axs[0].set_xlabel('Contrast Increase')
# 4F1
axs[1].plot(crfuns[2,:], 'o',markersize = 8,  label = 'Pre 4F1', color = x[2])
axs[1].plot(naka_rushton(contrast_levels, n_optimal[2], C50_ind[2], rMax_ind[2]),linewidth=3,label=f'Naka-Rushton, C50={C50_ind[2]:.4f}, Rmax={rMax_ind[2]:.4f}, nFit={n_optimal[2]:.4f}', color = x[2])
axs[1].plot(crfuns[3,:], 'o', markersize = 8,  label = 'Post 4F1', color = x[3])
axs[1].plot(naka_rushton(contrast_levels, n_optimal[3], C50_ind[3], rMax_ind[3]),linewidth=3,label=f'Naka-Rushton, C50={C50_ind[3]:.4f}, Rmax={rMax_ind[3]:.4f}, nFit={n_optimal[3]:.4f}', color = x[3])
axs[1].set_title('4F1 Naka Rushton Sim')
axs[1].legend(fontsize = 8, loc = 'lower right')
axs[1].spines['top'].set_visible(False)
axs[1].spines['right'].set_visible(False)
axs[1].set_ylabel('Response (R)')
axs[1].set_xlabel('Contrast Increase')
plt.suptitle('Conditional CRFs & Corresponding Hyperbolic Ratio Model', fontsize=18)
plt.show()
plt.close()
#print(C50_ind)
#print(rMax_ind)

#### Shuffle Across Trials - Pilot Bootstrap
#np.random.seed(10092006)
#shuff_data = np.zeros(t.shape)
#np.shape(shuff_data)
#shuff_trials = np.random.permutation(np.arange(NumTrials))
#shuff_data = t[:,:,shuff_trials]
#print(shuff_trials)
# check if numbers are shuffled
#print(t[1,1,0:5])
#print(shuff_data[1,1,0:5])
#t = shuff_data

In [ ]:
#data = fftData.rcaResult.projectedData; % 55 x 1
#weights = fftData.rcaResult.W; % 128 x 4 
#noise_l = fftData.rcaResult.noiseData.lowerSideBand; % 55 x 1
#noise_h = fftData.rcaResult.noiseData.higherSideBand; % 55 x 1